In [3]:
import os
import boto3
import boto3
import json
from dotenv import load_dotenv
from botocore.client import Config

In [4]:
# Load credentials
with open("credentials2.json") as f:
    creds = json.load(f)

# Load from .env file
load_dotenv()

endpoint = os.getenv('MINIO_ENDPOINT')
access_key = os.getenv('MINIO_ACCESS_KEY')
secret_key = os.getenv('MINIO_SECRET_KEY')
bucket_name = os.getenv('MINIO_BUCKET')

# ======= Khởi tạo S3 Client ========
s3_client = boto3.client(
    "s3",
    endpoint_url=endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version="s3v4"),
    # region_name="us-east-1"
)

def list_objects(bucket_name, prefix=""):
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    if "Contents" in response:
        for obj in response["Contents"]:
            print(obj["Key"])
    else:
        print(f"⚠️ Không tìm thấy object với prefix '{prefix}'")
    
def upload_to_s3(local_path, bucket, s3_prefix = ''):
    if os.path.isfile(local_path):
        s3_client.upload_file(local_path, bucket, s3_key)
        print(f"--> Đã upload {local_path} lên s3://{bucket}/{s3_key}")
        
    elif os.path.isdir(local_path):
        for root, dirs, files in os.walk(local_path):
            for file in files:
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, local_path)
                s3_key = os.path.join(s3_prefix, relative_path).replace("\\", "/").replace("//", "/")
                s3_client.upload_file(file_path, bucket, s3_key)
        print(f"--> Đã upload {local_path} lên s3://{bucket}/{s3_prefix}")
    
    else:
        print(f"The {local_path} does not exist!")

In [32]:
def list_folder_like_structure(bucket_name, prefix=""):
    response = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=prefix,
        Delimiter="/"
    )

    print("📁 Thư mục con:")
    for cp in response.get("CommonPrefixes", []):
        print(f" - {cp['Prefix']}")

    print("\n📄 File:")
    for obj in response.get("Contents", []):
        print(f" - {obj['Key']}")

In [5]:
item_metadata_folder = 'item_metadata'
safe_folder = 'GRD'
path = 'SENTINEL-1/C-SAR/GRD/'
for name in os.listdir(safe_folder)[1:2]:
    if name.lower().endswith('.safe'):
        print(name)
        safe_path = os.path.join(safe_folder, name)

        s3_prefix = f"{path}/{name}"
        upload_to_s3(safe_path, 'eov-platform-test', s3_prefix)

S1A_IW_GRDH_1SDV_20220415T224413_20220415T224438_042790_051B6B_2D71.SAFE
--> Đã upload GRD/S1A_IW_GRDH_1SDV_20220415T224413_20220415T224438_042790_051B6B_2D71.SAFE lên s3://eov-platform-test/SENTINEL-1/C-SAR/GRD//S1A_IW_GRDH_1SDV_20220415T224413_20220415T224438_042790_051B6B_2D71.SAFE
